<a href="https://colab.research.google.com/github/anitamezzetti/ML_finance/blob/main/generate_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
from time import time
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
start = time()

# Set the constants
# -----------------------------------------------------------------------------------
r = 0
S0 = k = 2.0
t_max = 1.0
V0 = 0.010201
sigma = 0.61
theta = 0.019
kappa = 6.21

rho_choice = np.array([-0.5, -0.7, -0.9])
rho_probability = np.array([0.25, 0.5, 0.25])

Stock prices simulation:

In [8]:
def stock_price_generator (t_max, n ,m, r, S0, k, V0, sigma, theta, kappa, rho_choice, rho_probability):
    dt = t_max / n
    
    # Brownian motions:
    dw_v = np.random.normal(size=(m, n)) * np.sqrt(dt)
    dw_i = np.random.normal(size=(m, n)) * np.sqrt(dt)

    rho = np.random.choice(rho_choice, size=(m, 1), p = rho_probability)
    dw_s = rho * dw_v + np.sqrt(1.0 - rho ** 2) * dw_i

    # Perform time evolution 
    s = np.empty((m, n + 1)) # initialisation stock prices vector
    s[:, 0] = S0

    v = np.ones(m) * V0

    for t in range(n):
        dv = kappa * (theta - v) * dt + sigma * np.sqrt(v) * dw_v[:, t]
        ds = r * s[:, t] * dt + np.sqrt(v) * s[:, t] * dw_s[:, t]

        v = np.clip(v + dv, a_min=0.0, a_max=None)
        s[:, t + 1] = s[:, t] + ds
        
        
    return s
    

In [9]:
def find_expected_payoff(stock_path, k, r, t_max):
    payoff = max(stock_path[-1] - k, 0) # one payoff for each simulation
    c = payoff * np.exp(-r * t_max)     # in case r=0, this step is useless
    
    return c

In [10]:
time_maturity = [0.5, 1, 2, 5]
num_times = [100, 250, 500]
num_simulations = [500, 1000]

In [11]:
for t in time_maturity[:1]:
    for n in num_times[:1]:
        for m in num_simulations[:1]:
            s = stock_price_generator (t_max, n ,m, r, S0, k, V0, sigma, theta, kappa, rho_choice, rho_probability)

            payoff = []
            for stock_path in s:
                payoff = np.append (payoff, find_expected_payoff(stock_path, k, r, t_max))

In [20]:
from plotnine import ggplot, aes, geom_point, geom_line, ggtitle

s = np.asmatrix(s)

df = pd.DataFrame({'X':np.squeeze(s[:,10]), 'Y':payoff})

(ggplot(df) + 
    geom_point(aes(x='X', y='Y'), alpha=0.05) +
    geom_line(aes(x='X', y='bs'), color="red") +
    ggtitle("Simulated call option payoffs and exact B&S pricing function")
)

Exception: ignored

In [30]:
np.squeeze(s[:,10])

matrix([[1.93981083, 1.98134586, 2.03280421, 2.04635878, 2.0368671 ,
         2.10640902, 1.91555365, 1.8409577 , 2.01912579, 2.08570044,
         1.94676407, 1.91494759, 2.03873874, 2.03368895, 1.94546799,
         2.06623008, 2.0225271 , 2.03395452, 1.82937072, 2.04379053,
         2.10113822, 2.04296748, 2.04646946, 2.01769641, 1.92452951,
         1.96097327, 1.96963606, 2.03863379, 1.95251778, 1.95198148,
         2.03114279, 1.94458956, 1.96118136, 1.83595799, 2.05272431,
         2.05807734, 2.01109881, 2.0183055 , 2.07967322, 1.93522612,
         2.00695881, 2.02035855, 1.81625346, 2.01278176, 2.04401678,
         2.01890867, 2.06288065, 2.13368876, 1.90821098, 2.06109229,
         2.0108477 , 2.0295212 , 2.03191766, 1.93923153, 2.11121659,
         1.98134113, 1.91754298, 2.06238166, 2.00810966, 2.01583107,
         1.86289884, 2.04403745, 2.01587335, 1.96870396, 1.90807378,
         2.10723821, 2.02317394, 2.04922736, 2.04586108, 1.93087314,
         2.02174694, 2.05289325, 1